<a href="https://colab.research.google.com/github/Jamalun/Data_Capstone_Design/blob/master/Sentimental_Dictionary_Trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 감성사전 제작 시도

금융뉴스 약 33만개의 문장에서 명사만을 추출해 빈도수를 카운팅했습니다. 

최다 빈출 명사를 내림차순으로 정렬해봤고,긍/부정을 분류하기 어려운 단어, 또는 중립 의미를 가진 단어가 대다수를 차지했습니다. 

이는 당연한 결과로 볼 수 있는데, 이 문제를 해결하기 위해 선행연구 '기업 재무분석을 위한 한국어 감성사전 구축'에서는 본인들의 주관적 판단과 기존 감성사전을 기준으로 긍/부정으로 분류했다고 합니다. 

하지만 전 제 개인적 주관으로 긍/부정을 분류하는 것은 의미가 없을 것이라 생각하기에, 기존에 진행하던 딥러닝 모델을 더 개선하려 합니다. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
## 약 33만개 금융 뉴스 문장 load
# 엑셀 파일 불러오기, header = none
df = pd.read_excel("/content/drive/MyDrive/Colab/selected_financial_sentence.xlsx",header=None)

In [4]:
df.head()

,0
0,중국 투자의향기업 직접 방문·투자협약 체결 협의
1,여수광양항만공사(사장 방희석)는 14일부터 20일까지 광양항 배후단지 투자 유치를 ...
2,광양만권경제자유구역청과 합동으로 진행되는 이번 투자유치활동은 방희석 사장이 직접 중...
3,방 사장은 먼저 광양항 서측배후단지 푸드존 투자유치를 위해 중국 운남성 소재 커피 ...
4,이어 한중일 콜드체인 거점 구축을 위해 중국 안후이성 소재 냉동냉장창고 업체인 바이...


# Mecab 설치

In [5]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 91 (delta 0), reused 0 (delta 0), pack-reused 90
Unpacking objects: 100% (91/91), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4 MB 1.4 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 448 kB 51.6 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-11-19 04:11:43--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22cd:e0db, 2406:da00:ff00::22e9:9f55, ...
Connecti

# 데이터 정제하기

In [6]:
# 정규 표현식 사용하여 한국어 이외 모두 제거
import re
import numpy as np
df.iloc[:,0] = df.iloc[:,0].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df.replace('', np.nan, inplace=True)
df = df.dropna(how='any') # Null 값 제거, 'how'= any 행에서 하나라도 결측치가 있다면 행삭제
print(df.isnull().sum())

0    0
dtype: int64


In [7]:
from konlpy.tag import Mecab
mecab = Mecab()

sample = "나도 삼성전자 한 주나 사려고 하는데, 너 생각은 어때?"
mecab.nouns(sample)

['나', '삼성전자', '주', '너', '생각']

In [8]:
from konlpy.tag import Mecab
mecab = Mecab()

noun_total = {}

for text in df.iloc[:,0]:
  noun_list = mecab.nouns(text)
  for noun in noun_list:
    noun_total[noun] = noun_total.get(noun, 0) + 1


In [9]:
noun_total

{'중국': 10292,
 '투자': 17824,
 '향': 468,
 '기업': 30734,
 '방문': 2965,
 '협약': 2704,
 '체결': 3037,
 '협의': 1630,
 '여수': 399,
 '광양항': 100,
 '공사': 4418,
 '사장': 3605,
 '방희석': 13,
 '일': 55872,
 '배후': 286,
 '유치': 1482,
 '참여': 6680,
 '활동': 5248,
 '중남부': 7,
 '지역': 21101,
 '광양만': 34,
 '경제자유구역': 137,
 '청과': 52,
 '합동': 453,
 '진행': 12909,
 '이번': 14619,
 '현지': 3980,
 '단지': 3058,
 '장점': 1088,
 '등': 79361,
 '소개': 1751,
 '방식': 3532,
 '방': 1012,
 '서측': 19,
 '푸드': 1169,
 '존': 724,
 '운남성': 1,
 '소재': 4441,
 '커피': 1178,
 '원재료': 297,
 '공급': 6531,
 '업체': 9247,
 '운남': 3,
 '메이': 134,
 '격치': 1,
 '주': 5786,
 '예정': 11450,
 '한중일': 68,
 '콜드': 64,
 '체인': 310,
 '거점': 669,
 '구축': 4238,
 '안후이': 1,
 '냉동': 346,
 '냉장창고': 4,
 '바': 2986,
 '식품': 5106,
 '입주': 2285,
 '계획': 13106,
 '지난해': 16823,
 '월': 32911,
 '경제': 12119,
 '청': 913,
 '선정': 5577,
 '후속': 392,
 '조치': 2547,
 '실질': 977,
 '수': 49071,
 '것': 57022,
 '성과': 3305,
 '기대': 6438,
 '배경': 582,
 '설명': 7732,
 '산업': 14370,
 '클러스터': 543,
 '발전': 3970,
 '계기': 1216,
 '앞': 5119,
 '활성': 3371,

In [12]:
sorted_dict = sorted(noun_total.items(), reverse = True, key = lambda item : item[1])
sorted_dict

[('등', 79361),
 ('것', 57022),
 ('일', 55872),
 ('원', 54884),
 ('수', 49071),
 ('년', 48639),
 ('만', 35689),
 ('월', 32911),
 ('기업', 30734),
 ('시장', 24650),
 ('억', 24214),
 ('사업', 24168),
 ('중', 22946),
 ('말', 22473),
 ('지역', 21101),
 ('이', 21056),
 ('개', 19422),
 ('금융', 19081),
 ('투자', 17824),
 ('지원', 17505),
 ('올해', 16914),
 ('지난해', 16823),
 ('명', 16796),
 ('한국', 16097),
 ('기술', 15739),
 ('개발', 14970),
 ('이번', 14619),
 ('산업', 14370),
 ('가능', 14290),
 ('제품', 14139),
 ('국내', 13854),
 ('서비스', 13615),
 ('증가', 13384),
 ('계획', 13106),
 ('대비', 13083),
 ('진행', 12909),
 ('고객', 12594),
 ('경제', 12119),
 ('상품', 11829),
 ('이상', 11802),
 ('대표', 11700),
 ('판매', 11641),
 ('예정', 11450),
 ('관계자', 11084),
 ('정부', 10869),
 ('제공', 10868),
 ('가격', 10830),
 ('확대', 10822),
 ('미국', 10797),
 ('규모', 10601),
 ('기준', 10601),
 ('전망', 10373),
 ('관련', 10302),
 ('중국', 10292),
 ('수출', 10178),
 ('서울', 9774),
 ('주택', 9768),
 ('경우', 9721),
 ('간', 9667),
 ('성장', 9586),
 ('차', 9491),
 ('최근', 9485),
 ('대상', 9350),
 ('분기', 9277)